<a href="https://colab.research.google.com/github/dodobir/blooket_hack_improved_Glixzzy/blob/master/Copy_of_Solutions_AdvancedCNNTechniques_Section3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="#de3023"><h1><b>REMINDER MAKE A COPY OF THIS NOTEBOOK, DO NOT EDIT</b></h1></font>

# Fake it 'til you make it!


CNNs often require a LOT of data to train on, more data than a researcher or organization might have available.

However we can trick our model into thinking we have more data than we do! This notebook will involve using two advanced techniques to simulate additional data: data augmentation and transfer learning.

<font color=darkorange size=5>**BEFORE RUNNING CODE: Change Hardware Accelerator to GPU to train faster (Runtime -> Change Runtime Type -> Hardware Accelerator -> T4 GPU)**

In [ ]:
# @title # **Run this code cell to set up the notebook!**
# @markdown The data may take some time to load in, so feel free to move on to the next part in the meantime.

project = "histology" # @param ["Choose your dataset!", "bees", "histology", "beans", "malaria"]

import requests
from IPython.display import Markdown, display

import tensorflow_datasets as tfds
from tensorflow.image import resize_with_pad, ResizeMethod

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def ProjectDescription(project):
  display_str =  f"**[{project.capitalize()} Project Background Document]({article_url_dict[project]})** <br />"
  display_str += f"**[{project.capitalize()} Dataset Documentation]({dataset_documentation_url_dict[project]})** <br />"
  display(Markdown(display_str))
  response = requests.get(image_url_dict[project], stream=True)
  img = Image.open(response.raw)
  plt.imshow(img)
  plt.axis('off')
  plt.show()

def plot_metric(history, metric="accuracy", best_is_max=True, start_epoch=0, random_model_metric=None):
  # Get lists of accuracies over the epochs
  training_accuracy = history.history[metric][start_epoch:]
  validation_accuracy = history.history['val_' + metric][start_epoch:]

  # Find best epoch depending on whether max is the best for the metric
  if best_is_max:
    best_epoch = validation_accuracy.index(max(validation_accuracy))
  else:
    best_epoch = validation_accuracy.index(min(validation_accuracy))

  # Plot labels
  plt.title(metric.capitalize() + ' as Model Trains')
  plt.xlabel('Epoch #')
  plt.ylabel(metric.capitalize())

  # Plot lines
  plt.plot(training_accuracy, label='Train')
  plt.plot(validation_accuracy, label='Validation')
  plt.axvline(x=best_epoch, linestyle='--', color='green', label='Best Epoch')

  if random_model_metric is not None:
    plt.axhline(random_model_metric, linestyle='--',color='red', label='Chance')

  # Plot legend and show
  plt.legend()
  plt.show()

# URL dictionaries for the projects
article_url_dict = {
    "beans"     : "https://docs.google.com/document/d/19AcNUO-9F4E9Jtc4bvFslGhyuM5pLxjCqKYV3rUaiCc/edit?usp=sharing",
    "malaria"   : "https://docs.google.com/document/d/1u_iX2oDrEZ1clhFefpP3V8uwAjf7EUV4G6kq_3JDcVY/edit?usp=sharing",
    "histology" : "https://docs.google.com/document/d/162WhUE9KqCgq_I7-VvENZD2n1IVsbeXVRSwfJEkxAqQ/edit?usp=sharing",
    "bees"      : "https://docs.google.com/document/d/1PUB_JuYHi6zyHsWAhkIb7D7ExeB1EfI09arc6Ad1bUY/edit?usp=sharing"
}

image_url_dict = {
    "beans"     : "https://storage.googleapis.com/tfds-data/visualization/fig/beans-0.1.0.png",
    "malaria"   : "https://storage.googleapis.com/tfds-data/visualization/fig/malaria-1.0.0.png",
    "histology" : "https://storage.googleapis.com/tfds-data/visualization/fig/colorectal_histology-2.0.0.png",
    "bees"      : "https://storage.googleapis.com/tfds-data/visualization/fig/bee_dataset-bee_dataset_300-1.0.0.png"
}

download_url_prefix_dict = {
    "histology" : "https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Towards%20Precision%20Medicine/",
    "bees"      : "https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Safeguarding%20Bee%20Health/"
}

dataset_documentation_url_dict = {
    "beans"     : "https://www.tensorflow.org/datasets/catalog/beans",
    "malaria"   : "https://www.tensorflow.org/datasets/catalog/malaria",
    "bees"      : "https://www.tensorflow.org/datasets/catalog/bee_dataset",
    "histology" : "https://www.tensorflow.org/datasets/catalog/colorectal_histology",
}

# Load dataset
if project == "Choose your dataset!":
  print("Please choose your dataset from the dropdown menu!")

elif project == "beans":
  data,  info = tfds.load('beans', split='train[:1024]', as_supervised=True, with_info=True)
  feature_dict = info.features['label'].names
  images = np.array([resize_with_pad(image, 128, 128, antialias=True) for image,_ in data]).astype(int)
  labels = [feature_dict[int(label)] for image,label in data]

elif project == "malaria":
  data,  info = tfds.load('malaria', split='train[:1024]', as_supervised=True, with_info=True)
  images = np.array([resize_with_pad(image, 256, 256, antialias=True) for image,_ in data]).astype(np.uint8)
  labels = ['malaria' if label==1 else 'healthy' for image,label in data]

else:
  wget_command = f'wget -q --show-progress "{download_url_prefix_dict[project]}'
  !{wget_command + 'images.npy" '}
  !{wget_command + 'labels.npy" '}

  images = np.load('images.npy')
  labels = np.load('labels.npy')

  !rm images.npy labels.npy


# Original preprocessing code for datasets

# if project == "histology":
#   data,  info = tfds.load('colorectal_histology', split='train[:1024]', as_supervised=True, with_info=True)
#   feature_dict = info.features['label'].names
#   images = np.array([image for image,label in data]).astype(int)
#   labels = [feature_dict[int(label)] for image,label in data]

# if project == "bees":
#   data,  info = tfds.load('bee_dataset', split='train[:3200]', as_supervised=True, with_info=True)
#   data = [(image, label) for image,label in data if label['wasps_output']==0]
#   data1 = [(image, label) for image,label in data if label['varroa_output']==0][:500]
#   data2 = [(image, label) for image,label in data if label['varroa_output']==1][:500]
#   data = data1 + data2
#   images = np.array([image for image, _ in data]).astype(int)
#   labels = ['diseased' if label['varroa_output'] else 'healthy' for image,label in data]

# PART 0: Remaking your model

So that you can focus on advanced techniques in CNNs, we are going to help you out by:
(1) Setting up your features and labels
(2) Splitting up your data into training and testing
(3) Constructing a similar CNN as in the previous notebook

After running the following cell, you will have the variables ```X_train```, ```X_test```, ```y_train```, and ```y_test``` along with your initialized (but untrained!) model, ```cnn_model```.


In return for this favor, you must run a couple sanity checks to make sure the dimensions of your variables, and the number of layers in your model is roughly what you expect!

In [ ]:
#@title Run this cell to construct your model!
#@markdown Feel free to click "Show code" and edit the model with any improvements you may have made in Notebook 2!
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Conv3D, Flatten

# Using the get_dummies() function to one-hot encode your labels.
labels_ohe = np.array(pd.get_dummies(labels))

# Select your feature (X) and labels (y).
y = labels_ohe
X = images

# Split your data into training and testing.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

# Initialize your model
cnn_model = Sequential()


# Input layer
cnn_model.add(Input(shape=X_train.shape[1:]))

# First layer
cnn_model.add(Conv2D(8, (3,3), activation='relu', padding="same"))
cnn_model.add(MaxPooling2D((2, 2)))

# Second layer
cnn_model.add(Conv2D(16, (3,3), activation='relu', padding="same"))
cnn_model.add(MaxPooling2D((2, 2)))

# Third layer
cnn_model.add(Conv2D(32, (3,3), activation='relu', padding="same"))
cnn_model.add(MaxPooling2D((2, 2)))

# # Fourth layer
cnn_model.add(Conv2D(64, (3,3), activation='relu', padding="same"))
cnn_model.add(MaxPooling2D((2, 2)))


# Flattening layer
cnn_model.add(Flatten())

# Hidden (dense) layer with 32 nodes, and relu activation function.
cnn_model.add(Dense(32, activation='relu'))

# Final output layer that uses a softmax activation function.
cnn_model.add(Dense(len(set(labels)), activation='softmax'))

# Compile your model
metrics_to_track = ['categorical_crossentropy', 'accuracy']
cnn_model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=metrics_to_track)



### Exercise 0A

Examine your data to ensure our preprocessing worked as intended.

Print the dimensions of ```X_test```, ```y_test```, ```X_train``` and ```y_train```.



In [ ]:
### YOUR CODE HERE ###

In [ ]:
#@title Instructor Solution
print('Dim X_train:', X_train.shape)
print('Dim y_train:', y_train.shape)
print('Dim X_test:', X_test.shape)
print('Dim y_test:', y_test.shape)

### Exercise 0B
Visualize the first 3 images in both ```X_train``` and ```X_test```.

In [ ]:
### YOUR CODE HERE ###

In [ ]:
#@title Instructor Solution
print("X_train")
for i in range(3):
  plt.imshow(X_train[i])
  plt.show()


print("X_test")
for i in range(3):
  plt.imshow(X_test[i])
  plt.show()

### Exercise 0C
Print the first 3 values of y_train and y_test, along with their categorical labels. We've given you a function to help transform the one-hot-encoded labels back into categories.

In [ ]:
one_hot_encoding_to_label_dict = {np.argmax(ohe):label for ohe, label in zip(labels_ohe, labels)}


# This function takes in a vector, and outputs the label.
def ScoreVectorToPredictions(prob_vector):
  class_num = np.argmax(prob_vector) # Find which element in the vector has the highest score.
  class_name = one_hot_encoding_to_label_dict[class_num] # Figure out the label that corresponds to this element.
  return class_name, max(prob_vector) # Return the label as well as the probabilty that the model assigned to this prediction.



### YOUR CODE HERE ###

In [ ]:
#@title Instructor Solution

one_hot_encoding_to_label_dict = {np.argmax(ohe):label for ohe, label in zip(labels_ohe, labels)}


# This function takes in a vector, and outputs the label.
def ScoreVectorToPredictions(prob_vector):
  class_num = np.argmax(prob_vector) # Find which element in the vector has the highest score.
  class_name = one_hot_encoding_to_label_dict[class_num] # Figure out the label that corresponds to this element.
  return class_name # Return the label as well as the probabilty that the model assigned to this prediction.

print("y train")
for i in range(3):
  print(ScoreVectorToPredictions(y_train[i]))

print("y test")
for i in range(3):
  print(ScoreVectorToPredictions(y_test[i]))


## Exercise 0D

Use the `.summary()` method to look at the architecture of your model. How many convolutional layers does your model contain? How many total parameters/weights will it be learning?

In [ ]:
### YOUR CODE HERE ###

In [ ]:
#@title Instructor Solution
cnn_model.summary()

## Exercise 0E

Retrain your model using your training data. Train your model for 10 epochs. Be sure to include the testing data as your `validation_data` and plot the model training history using `plot_metric()`.

In [ ]:
### YOUR CODE HERE ###

In [ ]:
#@title Instructor Solution
history = cnn_model.fit(X_train, y_train,
                        validation_data=(X_test, y_test), epochs=10)
plot_metric(history)

# PART I: Data Augmentation



In data augmentation, we can use the images we already have to create more data. For example, we could blur, change the colors, or rotate images to simulate a new image of a dress, shoe, or shirt (or whatever type of images you are working with) that the model could theoretically encounter.

<img src="http://ai.stanford.edu/blog/assets/img/posts/2020-04-20-data-augmentation/thumbnail.png" width=500>

### Exericse 1A

Write a function to transform the first image of your dataset by flipping it  upside down. You can use the ```flipud()``` function.  Then take a look at the original and augmented image.

In [ ]:
from numpy import flipud

## A function to create an augmented image from an original.
def createAugmentedImage(original_image):
  new_image = original_image ##### YOUR CODE HERE ########
  return new_image

# Transform the first image of your dataset using your new function.
new_image = createAugmentedImage(X_train[0])

# Let's see how your augmented image compares to the original!
# We've filled this part in for you.
f, ax = plt.subplots(ncols=2)
ax[0].imshow(X_train[0])
ax[0].set_title('New Image')
ax[1].imshow(new_image)
ax[1].set_title('Augmented Image')
plt.show()

In [ ]:
#@title Instructor Solution
from numpy import flipud

def createAugmentedImage(original_image):
  new_image = flipud(original_image)
  return new_image

# Transform the first image of your dataset using your new function.
new_image = createAugmentedImage(X_train[0])


# Let's see how your augmented image compares to the original!
# We've filled this part in for you.
f, ax = plt.subplots(ncols=2)
ax[0].imshow(X_train[0])
ax[0].set_title('New Image')
ax[1].imshow(new_image)
ax[1].set_title('Augmented Image')
plt.show()

### ⭐ BONUS ⭐

Using a different image transformation method to create an augmented image.   You can reuse the code above. **Note: Make sure you find an image transformation method that maintains the dimensions of your image! Otherwise it will no longer fit into your CNN.**

In [ ]:
### YOUR CODE HERE ###

### Exercise 1B

Using the first 100 images in your training dataset, and your ```createAugmentedImage()``` function, create an augmented dataset.  

You'll need to also get the associated labels for your augmented dataset. (Note: do not augment the y-values/labels at all, we want these to stay the same!)


In [ ]:
for i in range(100):
  new_X = None ### YOUR CODE HERE - What will you do to each image?
  new_y = None ### YOUR CODE HERE

  if i == 0:
    X_train_augment = [new_X]
    y_train_augment = [new_y]
  else:
    X_train_augment = np.append(X_train_augment, [new_X], axis=0)
    y_train_augment = np.append(y_train_augment, [new_y], axis=0)


print("Dimensions of augmented X:", X_train_augment.shape)
print("Dimensions of y:", y_train_augment.shape)

In [ ]:
#@title Instructor Solution
for i in range(100):
  new_X = createAugmentedImage(X_train[i])
  new_y = y_train[i]

  if i == 0:
    X_train_augment = [new_X]
    y_train_augment = [new_y]
  else:
    X_train_augment = np.append(X_train_augment, [new_X], axis=0)
    y_train_augment = np.append(y_train_augment, [new_y], axis=0)


print("Dimensions of augmented X:", X_train_augment.shape)
print("Dimensions of y:", y_train_augment.shape)

## Exercise 1C

Train your model (```cnn_model```) for 5 additional epochs using your new augmented dataset.

Use your testing data as validation to look at model performance as it trains.

Note: you do not need to start over with training, just start from where the model left off.

Does the augmented dataset improve performance? Why or why not?

In [ ]:
### YOUR CODE HERE ###

In [ ]:
#@title Instructor Solution
additional_history = cnn_model.fit(X_train_augment, y_train_augment, validation_data=(X_test, y_test), epochs=5)
plot_metric(additional_history)

# PART II: Transfer Learning




<img src="https://upload.wikimedia.org/wikipedia/commons/a/ab/Transfer_Learning.png" width=500>

No matter what the image classification task, we expect certain **geometric features** to be important - lines, edges, shapes, etc. We also expect convolutional neural networks to learn to recognize these shapes (most likely in the earlier layers) throughout their training. Therefore, we might be able to re-use the layers/weights (that presumably encode for recognizing lines, edges, etc.) that one image classification task has learned to perform another.

Many groups have published large neural networks trained on millions of images for classification tasks - Resnet, AlexNet, and MobileNet are a few famous ones. In order to perform transfer learning we can
1. Download one of these pretrained models - their weights have already been learned.
2. Change the output layer to have the number of nodes appropriate for our classification task (# of classes). Optionally, change the structure of any additional later layers you might want to alter.
3. Freeze the weights of most of the layers, especially the initial first few layers. This is an option you can specify before training the model, and will prevent model training from changing any of those weights. Don't freeze the last layer.
4. Alter your training images so that they are the right size for the input size of the downloaded neural network.
5. Train the model on your images/labels!


## 📶 Initialize a pre-trained model

### Exercise 2A

For our transfer learning, we're going to use 'experts' built upon the famous 'ImageNet' classification problem.

In ImageNet, participants were challenged to build machine learning models that could distinguish 14 million images' categories, where there were > 20,000 categories available.

Below, we see examples of 3 different categories.

![](https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/ImageNet.jpg)

This model will already have some object recognition expertise baked into it! We'll utilize this expertise by loading in this model, and we'll adapt it to our problem by replacing the last layer with a layer that we'll train ourselves.


Run the next cell to perform step 1, intializing one this large model.

Additionally, look at the summary of the model and answer:
- What is the input size of the images that VGG16 can classify?
- How many different categories of images does VGG16 classify?
- How many different weights (parameters) did VGG16 learn when it was originally trained?

In [ ]:
# Run this cell to import pretrained MobileNet
from keras.applications import MobileNetV2, VGG16
mobile_net = VGG16(include_top=True)
mobile_net.summary()

## 🧰 Modifying the output layer size

### Exercise 2B

Perform Step (2), rewiring the network to have an output layer with the # of nodes appropriate for our classification task.

After doing this, look at the final layer in your model. Does it have the number of neurons that you would like?

In [ ]:
from keras import Model


# Make a final layer. Use a Dense() layer, with the size as
# the number of of classes you want to predict!
# Also set the activation function to softmax.
new_output_layer = mobile_net.layers[0] ### REPLACE THIS LINE ###


# Rewire the model so that the new output layer
# The syntax here can be a little confusing, so we've helped you out.
output = new_output_layer(mobile_net.layers[-2].output)
input = mobile_net.input
transfer_cnn = Model(input, output)


# print the summary
transfer_cnn.summary()

In [ ]:
#@title Instructor Solution
from keras import Model

# Make a final layer. Use a Dense() layer, with the size as
# the number of of classes you want to predict!
# Also set the activation function to softmax.
new_output_layer = Dense(len(y_train[0]), activation='softmax') ### REPLACE THIS LINE ###

# Rewire the model so that the new output layer
# The syntax here can be a little confusing, so we've helped you out.
output = new_output_layer(mobile_net.layers[-2].output)
input = mobile_net.input
transfer_cnn = Model(input, output)

# print the summary
transfer_cnn.summary()

## 🧊 Freezing the weights

### Exercise 2C

Next, freeze the weights for all of the layers except for the final layer. You can do this by setting
```layer.trainable=False``` where ```layer``` is the layer you are interested in freezing (``trainable=False``) or unfreezing (``trainable=True``) the weights of.

Additionally, compile your model and look at its summary.
 Now, how many trainable parameters (weights) are there compared to before?

In [ ]:
# make all layers untrainable by freezing weights (except for last layer)
for layer in transfer_cnn.layers:
    layer.trainable = True ### YOUR CODE HERE - Make sure all layers are >>>NOT<<<< trainable

## Set the final layer as trainable=True
### YOUR CODE HERE ###


# Compile your new model using loss='categorical_crossentropy'
# optimizer='adam' and metrics=['accuracy', 'categorical_crossentropy']
### YOUR CODE HERE ####

# Look at the summary of the network to make sure the structure is as you expect!
transfer_cnn.summary()

In [ ]:
#@title Instructor Solution

# make all layers untrainable by freezing weights (except for last layer)
for layer in transfer_cnn.layers:
    layer.trainable = False

## Set the final layer as trainable=True
transfer_cnn.layers[-1].trainable = True

# Compile your new model using loss='categorical_crossentropy'
# optimizer='adam' and metrics=['accuracy', 'categorical_crossentropy']
transfer_cnn.compile(loss='categorical_crossentropy', optimizer='adam',
                     metrics=['accuracy', 'categorical_crossentropy'])
# Look at the summary of the network to make sure the structure is as you expect!
transfer_cnn.summary()

## ✂ Resize your images

### Exercise 2D

Resize your images so that they are the appropriate size for the input layer of your model. You can use the function we have given you, which takes in a set of ```images``` and resizes them to have the appropriate ```heights``` and ```weights```.



In [ ]:
# Takes in an image, a new height, and a new width
# and resizes the image, plus converts from greyscale to 3 RGB color channels.
def ResizeImages(images, height, width):
  return np.array([resize_with_pad(image, height, width, antialias=True) for image in images]).astype(int)

# Resize your image
X_train_resized = X_train ### REPLACE THIS LINE
X_test_resized = X_test ### REPLACE THIS LINE

# Make sure your images are the right dimensions
print("Dim X_train_resized:", X_train_resized.shape)
print("Dim X_test_resized:", X_test_resized.shape)

In [ ]:
#@title Instructor Solution

def ResizeImages(images, height, width):
  return np.array([resize_with_pad(image, height, width, antialias=True) for image in images]).astype(int)

X_train_resized = ResizeImages(X_train, 224, 224)
X_test_resized = ResizeImages(X_test, 224, 224)

# Make sure your images are the right dimensions
print("Dim X_train_resized:", X_train_resized.shape)
print("Dim X_test_resized:", X_test_resized.shape)

## 💅 Finetune the model using your data

### Exercise 2E

Finally, train your model using your data. Because this model takes a while to train, just train for 3 epochs.

Use your testing data as validation to look at model performance as it trains.

In [ ]:
### YOUR CODE HERE ###

In [ ]:
#@title Instructor Solution
transfer_history = transfer_cnn.fit(X_train_resized, y_train, validation_data=(X_test_resized, y_test), epochs=10)
plot_metric(transfer_history)

# PART III: Wrapping Up




Congrats!!! You used some very advanced techniques to build a sophisticated machine learning model! If you didn't understand everything, ***don't worry!*** Transfer learning especially is a very modern field and is still being explored.

<img src="https://c.pxhere.com/photos/04/45/fireworks_celebration_bright_pink_explosive_celebrate_display_july_4th-1160027.jpg!d" width=500>



### Exercise 3A

To wrap up, answer the following questions:

(1) Did data augmentation or transfer learning work better to improve the performance of your model? Why do you think one worked better than the other?

(2) What might be a better type of data augmentation strategy to use for your data?

(3) What might be a better type of dataset to use for transfer learning?



## 🔭 Looking back on your project

Just like our neural networks are able to *learn* from data and improve performance, so can you!

Therefore, let's do a little retrospective for this project, and talk about what went well and what was challenging.

We'll do this in a fun little game called: rose-thorn-bud. Discuss with your team and with the rest of the class!

### Exercise 3B

🌹 Rose: What went well in the project in terms of...

- model performance? In what areas did your model perform well in?
- coding? What parts of the code did you feel like you mastered by the end of the project?
- teamwork? What methods did you use to collaborate with your teammates?

### Exercise 3C

🔺 Thorn: What was challenging in the project in terms of...

- model performance? In what areas did your model perform poorly?
- coding? What parts of the code did you feel like you could use more work on?
- teamwork? What was something your team struggled with during the project?

### Exercise 3D

🌱 Bud: What are some future directions for your project in terms of...

- model performance? If you had infinite time and resources, what strategies might you take to improve your model even more?
- coding? What parts of the coding or machine learning concepts would you like to dive deeper into?
- teamwork? What are some ways to work better as a team that you are excited to try next time you are in a group project?